In [ ]:
include("../src/StatReg.jl")
using .StatReg

Define target function $\varphi(x) = \frac{\sin(x)}{x}$ with support $[-20,20]$:

In [ ]:
using Plots
plotly()
gr(size=(500,500), html_output_format=:png)

a = -20.
b = 20.

function phi(x::Real)
    if x != 0
        return sin(x)/x
    else
        return 1
    end
end

x = collect(range(a, stop=b, length=300))
myplot = plot(x, phi.(x), title="Real phi function",label=["Phi(x)"])

We will use diffraction kernel $K(x, y)$. $f(y)$ is an observed function, we need it to find the target function.

In [ ]:
kernel(x::Real, y::Real) = getOpticsKernels("diffraction")(x, y)

using QuadGK
include("C:/Users/user/.julia/dev/StatReg/src/kernels.jl")
convolution = y -> quadgk(x -> kernel(x,y) * phi(x), a, b, rtol=10^-5, maxevals=10^7)[1]
y = collect(range(a, stop=b, length=30))
ftrue = convolution.(y)
myplot1 = plot(y, ftrue, label = "phi(x)")

Generation of noise (measurement errors):

In [ ]:
sig = ftrue*0.1 + [0.01 for i = 1:Base.length(ftrue)]

using Compat, Random, Distributions
noise = []
Random.seed!(1234)
for sigma in sig
    n = rand(Normal(0., abs(sigma)), 1)[1]
    push!(noise, n)
end

f = ftrue + noise
plot(y, f, title = "Integrated function", label = ["f(y)"], seriestype=:scatter, yerr=sig)
plot!(y, f, label = "f(y)")

We will use a Fourier Basis with 17 functions:

In [ ]:
basis = FourierBasis(a, b, 8)
p = plot()
for func in basis.basis_functions
    p = plot!(x, func.f.(x), title="Fourier basis functions", legend=false, show = true)
end
display(p)

In [ ]:
Omega = omega(basis, 2)
model = GaussErrorUnfolder(basis, [Omega], "EmpiricalBayes", nothing, [1e-8], [10.], [0.3])
println()

result = solve(model, kernel, f, sig, y)
println()

phivec = PhiVec(result, basis)

phi_reconstructed = phivec.phi_function.(x)
phi_reconstructed_errors = phivec.error_function.(x)

plot(x, phi_reconstructed, ribbon=phi_reconstructed_errors, fillalpha=0.3, label="Reconstructed function with errors")
plot!(x, phi.(x), label="Real function")